In [ ]:
#+ Get entities from wikipedia
#+ Get Pages from those entities
#+ Get abstract of whose pages
#+ Split abstact on passages (in wikipedia spliters can be  == Movement to Organize Black Psychologists == , or \n)
# use word2vec for comare passage and question
# return 1 most relevant passage to aqustion for page

In [4]:
import spacy
en_nlp=spacy.load('en_core_web_lg')

In [45]:
import spacy
from nltk import Tree 
import tagme
from operator import itemgetter
import wikipedia
wikipedia.set_lang("en")
tagme.GCUBE_TOKEN = "c294fe63-88fa-40e1-b1d4-34b70c887b29-843339462"
import pickle
import re
from sklearn.feature_extraction.text import HashingVectorizer
import numpy as np
import nltk, string
import joblib
from sklearn.metrics.pairwise import cosine_similarity

class text_classifier:
    # can be improved by using word vectors
    def __init__(self, data, vectorizer):
        self.vectorizer = vectorizer
        self.texts_vectors = self.vectorizer.vectorize_data(data)
        self.texts=data
        
    def get_cosines_index(self, text):
        #vectoriztion
        text_vector = self.vectorizer.vectorize_text(text)
        #dot similarity
        cosine = cosine_similarity(text_vector, self.texts_vectors)
        return cosine[0].argsort()[:-1]

    
    def get_jaccard_index(self, text):
        #vectoriztion
        distances = []
        for source_text in self.texts:
            distances.append(self.DistJaccard(text,source_text))
        return np.argsort(distances)[:-1]
    
    def DistJaccard(self, str1, str2):
        str1 = set(self.vectorizer.tokenize(str1))
        str2 = set(self.vectorizer.tokenize(str2))
        return len(str1 & str2)
    
    
class english_text_vectorizer:
    
    def __init__(self, folder_with_vectorizers, en_nlp):
        self.folder_with_vectorizers=folder_with_vectorizers
        self.en_nlp = en_nlp
        self.vectorizer = HashingVectorizer(tokenizer=lambda text:self.get_lemmas(text),n_features=100000)
        self.remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
        
        try:
            self.vect = joblib.load(folder_with_vectorizers+'en_HashingVectorizer.pkl')
            self.vect.tokenizer=self.text_to_lemmas
        except:
            print("HashingVectorizer was not loaded")
            self.vect = HashingVectorizer(tokenizer=lambda text:self.tokenize(text),n_features=100000)
        
    def tokenize(self, text):
        text = text.lower().translate(self.remove_punctuation_map)
        return [token.lemma_ for token in self.en_nlp(text)]
        
    def vectorize_data(self, texts):
        try:
            return self.vect.transform(texts)
        except:
            vectors = self.vect.fit_transform(texts)
            joblib.dump(self.vect, self.folder_with_vectorizers+'en_HashingVectorizer.pkl', compress=9)
            return vectors        
    
    def vectorize_text(self, text):
        return self.vect.transform([text])[0]

class cache_helper:
    
    def __init__(self, file_path):
        self.file_path = file_path
        self.dict = None  
    
    def get(self, key, get_value):
        if self.dict == None:
            self.load()
        if key not in self.dict:
            print('need to download')
            self.set_item(key, get_value(key))
        return self.dict[key]   
    
    def set_item(self, key, value):
        self.dict[key]=value
        self.save()
        
    def save(self):
        with open(self.file_path, 'wb') as f:
            pickle.dump(self.dict, f, pickle.HIGHEST_PROTOCOL)
            
    def load(self):
        try:
            with open(self.file_path, 'rb') as f:
                self.dict=pickle.load(f)
        except:
            self.dict={}
    
class WikiHelper:
    
    def __init__(self, en_nlp, cache_file):
        self.en_nlp=en_nlp
        self.data=cache_helper(cache_file)
        
    def get_spacy_entities(self, text, count=3):
        nlp_doc = self.en_nlp(text)
        return [ent.text for ent in nlp_doc.ents[:count]]
    
    def get_wiki_entities(self, text, count=3):
        newlist = [{'score': item.score,'mention':item.mention,'entity_title':item.entity_title} for item in tagme.annotate(text).get_annotations(0.1)]
        newlist = sorted(newlist, key=itemgetter('score'), reverse=True)
        return [{'mention':item['mention'],'entity_title':item['entity_title']} for item in newlist[:count]]
    
    def get_merget_entities(self, text):
        entities = self.get_wiki_entities(text)
        if len(entities)<2:
            spacy_entities =  self.get_spacy_entities(text)
            for spacy_entity in spacy_entities:
                for entity in entities:
                    if (spacy_entity not in entity['mention']) and (entity['mention'] not in spacy_entity):
                        entities.append({'mention':spacy_entity,'entity_title':spacy_entity})
        return entities
    
    def get_wiki_content(self, entity, pages_count):
        fined_page_names = wikipedia.search(entity)
        if len(fined_page_names)>0:
            try:
                content=[]
                for page_name in fined_page_names[:pages_count]:
                    wiki_page = wikipedia.page(page_name)
                    content.append(wiki_page.content)
                return '. '.join(content)
            except:
                return ''
        else:
            return ''
        
    def get_pages(self, text):
        print('wiki processing')
        wiki_content=[]
        entities = self.get_merget_entities(text)
        for entity in entities:
            wiki_content.append(self.get_wiki_content(entity['entity_title'],1))
        return wiki_content
    
    def get_data(self,text):
        texts = self.data.get(text, self.get_pages)
        return [self.split_text_to_passages(text) for text in texts]
    
    def split_text_to_passages(self, text):
        split_pattern = "(\\n)+|(=+ ([a-zA-Z]* )+=+)"
        return [passage for passage in  re.split(split_pattern, text) if (passage != None) and (len(passage.strip().split(' '))>1) and (passage != '\n') and (passage[0] != '=')]
    
    
class IR_Wiki:
    def __init__(self, en_nlp):
        self.vectorizer=english_text_vectorizer('vectorizers',en_nlp)
        self.wikiHelper = WikiHelper(en_nlp, 'WikiHelper.pkl')
        
    def get_most_relevant_passage(self, question, passages):
        classifier = text_classifier(passages, self.vectorizer)
        indexes = classifier.get_jaccard_index(question)        
        return np.array(passages)[indexes]
        
    def get_passage(self, question):
        data = self.wikiHelper.get_data(question)
        passages = np.concatenate(data)      
        return self.get_most_relevant_passage(question,passages)

In [46]:
ir = IR_Wiki(en_nlp)

HashingVectorizer was not loaded


In [60]:
pas = ir.get_passage('what is the capital of ukraine ?')

In [8]:
pas

array(["Hospitals in Ukraine are organised along the same lines as most European nations, according to the regional administrative structure; as a result most towns have their own hospital (Міська Лікарня) and many also have district hospitals (Районна Лікарня). Larger and more specialised medical complexes tend only to be found in major cities, with some even more specialised units located only in the capital, Kiev. However, all oblasts have their own network of general hospitals which are able to deal with almost all medical problems and are typically equipped with major trauma centres; such hospitals are called 'regional hospitals' (Обласна Лікарня).",
       "In 1934, the capital of Soviet Ukraine moved from Kharkiv to Kiev. Previously, the city was seen as only a regional centre, hence received little attention. All of that was to change, at great price. The first examples of Stalinist architecture were already showing, and, in light of the official policy, a new city was to be bu

# test performanse based on passages from training set

In [9]:
with open ('passages_test_query', 'rb') as fp:
    passages_test_query = pickle.load(fp)

In [13]:
passages_test_query[0]

'average cost to build large retaining wall'

In [10]:
with open ('passages_test', 'rb') as fp:
    passages_test = pickle.load(fp)

In [15]:
def select_passages_and_label(passage_group):
    passages=[]
    label = None
    for passage in passage_group:
        passages.append(passage['passage_text'])
        if (passage['is_selected']):
            label=passage['passage_text']
    return passages, label

In [18]:
passages, correct_passage = select_passages_and_label(passages_test[0])

In [27]:
np.array(passages)[[1,5]]

array(['1 Materials and installation for a stone retaining wall start around $25-$40 a square face foot and go up depending on materials, site conditions, and other factors. 2  It costs $1,250-$2,000 or more for a wall 30 inches high and 20 feet long.hopping for a retaining wall: 1  Hiring an engineer to help design a retaining wall can run about $200-$500 or more, depending on the complexity and extent of the project. 2  Hiring an engineer is strongly recommended for walls over 3 feet high.',
       "1 When the wall is increased to 8' in height the cost will average at $13,700, and when the height is bumped up to 10' the cost increases to $19,900.2  Estimates and quotes-like many other major construction projects it is advisable to seek out estimates from a range of providers.or the purpose of this discussion we will consider the costs of building a retaining wall from cast concrete and at a length of 50 linear feet. We will also take into consideration various heights as well, rangin

In [ ]:
len()

In [51]:
index = 3

passages, correct_passage = select_passages_and_label(passages_test[index])
predicted_passage = ir.get_most_relevant_passage(passages_test_query[index], passages)
print(passages_test_query[index],': ',correct_passage==predicted_passage)
print('+++++++++++++++++++++')
print(correct_passage)
print('+++++++++++++++++++++')
for p in predicted_passage:
    print(p)
    print('---------------------')

cortana what's agony :  [False False False False False False False False False]
+++++++++++++++++++++
None
+++++++++++++++++++++
noun, plural ironies. 1. the use of words to convey a meaning that is the opposite of its literal meaning: the irony of her reply, “How nice!” when I said I had to work all weekend. 2. Literature. a technique of indicating, as through character or plot development, an intention or attitude opposite to that which is actually or ostensibly stated.
---------------------
Micropenis: When a Penis Really Is Too Small. There is, of course, such a thing as a very small penis. The medical term micropenis applies to the 0.6% of men with the smallest penises. According to Palmer's statistics, an SPL of three and two-thirds inches or less indicates a micropenis.
---------------------
Boston crab. Chris Jericho with his Walls of Jericho (Elevated Boston crab) on Shawn Michaels. The Boston crab is a professional wrestling hold that typically starts with one wrestler lying 

In [ ]:
# there quite similar passages with information, but machine comprehension model somehow select from them

In [12]:
len(passages_test_query)

727226

In [58]:
results = []
from tqdm import tqdm
for i in tqdm(range(1000)):
    passages, correct_passage = select_passages_and_label(passages_test[i])
    predicted_passage = ir.get_most_relevant_passage(passages_test_query[i], passages)
    #print(list(correct_passage==predicted_passage))
    try:
        results.append(list(correct_passage==predicted_passage).index(True))
    except:
        results.append(-1)

100%|██████████| 1000/1000 [06:44<00:00,  2.47it/s]


In [59]:
from collections import Counter
Counter(results)

Counter({-1: 532,
         0: 32,
         1: 31,
         2: 34,
         3: 34,
         4: 52,
         5: 50,
         6: 76,
         7: 69,
         8: 90})